For Python programmers Ibis offers a way to write SQL in Python that allows for unit-testing, composability, and abstraction over specific query engines (e.g.BigQuery)! You can carry out joins, filters and other operations on your data in a familiar, Pandas-like syntax. Overall, using Ibis simplifies your workflows, makes you more productive, and keeps your code readable.

Let's start by getting ibis and sqlite installed on your system. In the next several cells, you will notice that the first character is an exclamation point: ***!*** This tells Jupyter that we're running a shell (terminal) command instead of python code in the notebook

## Install SQLite

In [ ]:
!conda install -c anaconda sqlite 

## Install ibis

In [ ]:
!conda install -c conda-forge ibis-framework     

## Get the data
We're using the `Civic Art Collection` database, which can be downloaded here: https://data.sfgov.org/Culture-and-Recreation/Civic-Art-Collection/r7bn-7v9c using the `Export` service in the top right corner. Please select `CSV` for your export format. When you've completed the download, store the downloaded file in the `data` directory in this folder.

When you're done, you should see the following information when you list this folder:
>`data                       ibis_sqlite_tutorial.ipynb`

and when you list the Data folder you should see: 
>`Civic_Art_Collection.csv`
<br>

## Create the SQLite database

### let's first create variables for all of the things we're about to go do

In [1]:
import os
path="../data" #where did we store the data
file="Civic_Art_Collection.csv" #what's the name of the file we're importing
collection_name="civicArt.db" # what are we going to call the database
table_name="civicArtTable" # what are we going to call the table into which we are importing the file
path_separator=os.path.sep # get the path separator (forward or backward slash) for this operating system
conn=None # place holder for the db connection

### Create the SQL Collection (database)
SQLite wants a file with the name of the collection as the db. so we'll use the shell `touch` command to create it. 

In [2]:
!touch "{collection_name}"

### Create the table schema
We will use pandas to help use create the schema and, later on, load the data into SQLite

In [3]:
import pandas as pd
type_dic = {"object": "String", "float64": "Float64", "int64": "Int64"}
art_df = pd.read_csv(path+path_separator+file).fillna(0.00)
col_names, col_types = [], []

for e in zip(art_df.columns, art_df.dtypes):
    col_names.append('_'.join(e[0].lower().split(' ')))
    col_types.append(type_dic[e[1].__str__()])
art_df.columns = col_names
# col_names, col_types

### Create the table
This defines where the data will go and it's 'shape' - that is, the characteristics of each column of data

In [4]:
import ibis
ibis.options.interactive = True
db = ibis.sqlite.connect(collection_name)

In [5]:
schema = ibis.Schema(names=col_names, types=col_types)
db.create_table(table_name, schema=schema)

### import the data into the table

In [6]:
db.load_data(table_name, art_df, if_exists='replace')

#### list the tables

In [7]:
sql_tables = db.list_tables()
print(sql_tables)

['civicArtTable']


#### list the columns in the table

In [8]:
# since the results of the preceding command is a list and we may get more than one table in the list
# print each table name and the columns for that table
for each in sql_tables:
    print(f"columns in {each}")
    print(f"{db.table(each).columns}")

columns in civicArtTable
['creation_date', 'accession_number', 'artist', 'credit_line', 'display_title', 'display_dimensions', 'medium', 'media_support', 'facility', 'current_location', 'location_description', 'street_address_or_intersection', 'zip_code', 'latitude', 'longitude', 'number_of_districts', 'cultural_districts', 'supervisor_district', 'the_geom', 'sf_find_neighborhoods', 'current_police_districts', 'current_supervisor_districts', 'analysis_neighborhoods', 'neighborhoods']


## Querying 

Anything you can write in a SELECT statement you can write in Ibis. Let's test this out!
I’ll use the following code to find out which artists have art currently displayed in the city and what the title of their pieces are. 

### Selecting columns from a table 

In [11]:
art = db.table(sql_tables[0])
sql_results = art[["artist", "display_title"]]
sql_results

,artist,display_title
0,"O'Malley, Susan",Advice from My 80-Year-Old Self: Live Each Day...
1,"Moya del Pino, Jose",Murals for the Acme Beer Company (The Gatherin...
2,"Ng, Win",Unknown
3,"Colburn, Adriane",Geologic Ghost
4,"Alexopoulos, Yorgo",Split
...,...,...
1033,"Bongers, Jaap (Jacob)",Untitled #2
1034,"Fuller (McChesney), Mary",Bench support (Tiger)
1035,"Ghidini, Sheila and Hunter, Leonard",19th Avenue Shelter at Holloway Avenue
1036,"deSoto, Lewis",1909: President Theodore Roosevelt Visits the ...


### Filtering Data

Next let's pick an artist and figure out where exactly all their art is located. Adriane Colburns’ display title, `Geological Ghost` caught my eye so let's choose them! 

I use the following commands to do this

In [12]:
adrianes_art = art.filter(art.artist == 'Colburn, Adriane')
adrianes_art

,creation_date,accession_number,artist,credit_line,display_title,display_dimensions,medium,media_support,facility,current_location,...,longitude,number_of_districts,cultural_districts,supervisor_district,the_geom,sf_find_neighborhoods,current_police_districts,current_supervisor_districts,analysis_neighborhoods,neighborhoods
0,2020,T2016.046,"Colburn, Adriane",0.0,Geologic Ghost,0.0,waterjet cut steel,0.0,Guy Place Mini Park,City Agency,...,-122.394391,0,0.0,6.0,POINT (-122.39439100099997 37.78668401700003),30.0,1.0,10.0,8.0,30.0
1,2016,2017.1.a-e,"Colburn, Adriane",Collection of the City and County of San Franc...,Shadow Kingdom,0.0,Painted Steel and granite,Painted Steel and granite,Daggett Park,Public Display,...,-122.396320,0,0.0,10.0,POINT (-122.39631992899997 37.766470051000056),33.0,1.0,9.0,4.0,33.0


So it turns out Adriane has two pieces on display, one at `4 Guy Place` and the other at `Dagget Street & 16th Street`. This is great, we already have some places we can add to our tourist itinerary!  

### Groupby

I don’t usually stay more than one or two days in a city after a conference, so it might be nice to know which locations have the most art on display. To figure this out we’ll use the following `groupby` expressions to get the information we need. 
We use `groupy` and `sort_by` to get the 10 locations in San Francisco with the most art! 

In [22]:
most_art = (
    art
    .filter(art.street_address_or_intersection !="0.0") # remove outliers
    .groupby("street_address_or_intersection")          # aggregate by street
    .count('display_title')
    .sort_by(ibis.desc('display_title'))                # sort by count in a descending order
    .limit(10)                                          # take top 10
)
most_art


,street_address_or_intersection,display_title
0,1001 Portrero Avenue,59
1,1 Telegraph Hill Boulevard,38
2,2695 19th Avenue,28
3,100 Collingwood Street,25
4,236 Monterey Boulevard,25
5,Stern Grove Entry Road & Wawona Street,24
6,375 Laguna Honda Boulevad,23
7,55 S McDonnell Road,23
8,Clay Street & Kearny Street,22
9,3090 23rd Street,18


#### Great, the first stop would be 1001 Potrero Avenue that has 59 titles on display!! 